This doc run the data refresh process for the US48 demand for electricity dataset from the EIA API. It includes the following functionality:

- Check if new data available on the API
- Setting the data request parameters and pull the data
- Data quality checks 
- Appending the new data and saving the data
- Updating the log file 

## Load libraries

We will pull the data from the EIA API using a set of functions on the `eia_api.py` file. This includes the following functions:

- `eia_get` - A function for query data from the API. Can pull up to 5000 rows per call
- `eia_backfile` - A wrapper function, using batches requests from the API using the `eia_get` function to serve large requests (more than 5000 rows)
- `day_offset` - A helper function creates a vector of dates equally spaced by days
- `hour_offset` - A helper function creates a vector of dates equally spaced by days

The `eia_api.py` file imports the following libraries:

- `pandas` - for data processing
- `datetime` - to work with dates and time objects
- `requests` - to send `GET` requests to the EIA API


In addition, we will use the following libraries:

- `os` - load environment variables
- `numpy` - to create sequences (vectors)
- `plotly` - visualize the data
- `pathlib` - set file path
- `ydata_profiling` - for data profiling


In [ ]:
#| label: load libraries
import eia_api
import eia_etl as etl
import eia_forecast as fc
import os
import datetime
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from statistics import mean
from pathlib import Path
from ydata_profiling import ProfileReport
from zoneinfo import ZoneInfo
from darts import TimeSeries
from darts.models.forecasting.linear_regression_model import LinearRegressionModel

## Load the Log File and API Metadata

We will load the log file from the `data` folder:


In [ ]:
log_file = etl.load_log(path = "data/us48_metadata.csv")

The `load_log` function returns the following objects:

- `log` - the full data log
- `last_success` - the log of the last success data pull
- `end` - the timestamp of the last data point derived from the `last_success` row 
- `start` - the start timestamp of the future data pull (calculated as `end` + 1 hour)


In [ ]:
print(log_file.last_success)
print(log_file.end)
print(log_file.start)

start = log_file.start

Setting the GET request parameters:


In [ ]:
api_path = "electricity/rto/region-data/data"
facets = {
    "respondent": "US48",
    "type": "D"
}

offset = 2250

eia_api_key = os.getenv('EIA_API_KEY')

Pulling the series metadata from the API. We will use it to check if new data is available on the API using the `get_api_end` function:


In [ ]:
metadata = etl.get_api_end(api_key = eia_api_key, 
api_path = "electricity/rto/region-data/", offset = 16)

The `end` variable represents the timestamp of the most recent data point available on the API. There is some issue on the API timestampe, a gap of 16 hours between the value of the last data point and the actual, therefore, we will offset the `end` value by 16 hours using the `offset` argument and save it as `end_fix`:


In [ ]:
print(metadata.end)
print(metadata.end_fix)

end_fix = metadata.end_fix

In [ ]:
new_data = etl.eia_data_refresh(start = start, end = end_fix, api_key = eia_api_key, api_path = api_path, facets = facets) 

## Updating the Dataset and Log File


In [ ]:
append_data = etl.append_new_data(data_path = "data/us48.csv",
                           log_path = "data/us48_metadata.csv",
                           new_data = new_data,
                           save = True)

data = append_data.data

In [ ]:
print(append_data.log)
print(append_data.data_update)

## Forecast Refresh

Forecast scope:
- Used a train forecasting model and refit it with new data
- Forecast refresh daily at the end of the day (past 23:00 UTC)
- Forecast horizon 24 hours

Steps:
- Load the forecast log file
- Check the time stamp of the most recent forecast
- Check if new data meet the refresh criteria
- If meet the criteria, update the forecast
- Update the log file



Forecast parameters


In [ ]:
h = 24
freq = 24
num_sample = 500 

In [ ]:
log_path = "data/fc48_metadata.csv"
fc_path = "data/fc48.csv"

forecast_start = data["period"].max().floor(freq = "d")
last_start = fc.get_last_fc_start(log_path = log_path)

if last_start < forecast_start:
    end = forecast_start - datetime.timedelta(hours = 1)
    start = end - datetime.timedelta(hours = freq * 365 * 2)
    print(start, end)
    input = fc.set_input(input = data, start = start, end = end)
    
    forecast = fc.train_lm(input = input, 
            lags = [ -freq, -7 * freq,  - 365 * freq],
            likelihood = "quantile",
            quantiles = [0.025, 0.1, 0.25, 0.5, 0.75, 0.9, 0.975],
            h = h,
            pi = 0.95,
            num_samples = 500)
    print(forecast.log)
    log = fc.append_log(log_path= log_path, new_log = forecast.log, save = True, init = False)
    new_fc = fc.append_forecast(fc_path =  "data/fc48.csv", fc_new = forecast, save = True, init = False)

Score past forecasts


In [ ]:
fc = fc.load_forecast(fc_path)
new_log = fc.score_forecast(log_path = "data/fc48_metadata.csv", actual = data, forecast = new_fc, save = True)

In [ ]:
log_path = "data/fc48_metadata.csv"

# def get_last_fc_start(log_path):
    
fc_log = pd.read_csv(log_path)
fc_log = fc_log[fc_log["success"] == True]
fc_log = fc_log[fc_log["index"] == fc_log["index"].max()]
fc_log["start_act"] = pd.to_datetime(fc_log["start_act"])
fc_log["end_act"] = pd.to_datetime(fc_log["end_act"])

    # return fc_log
    
last_start = get_last_fc_start(log_path = log_path)
# print(last_start)

# forecast_start = data["period"].max().floor(freq = "d")

# if last_start < pd.to_datetime(forecast_start):
#     print("test")

Forecast model:

In [ ]:
lr_model = LinearRegressionModel(lags= [ -freq, -7 * freq,  - 365 * freq],
                                 likelihood= "quantile", 
                                 quantiles = [0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95])


In [ ]:
fc_meta = pd.read_csv("data/fc48_metadata.csv")
fc_meta["start_act"] = pd.to_datetime(fc_meta["start_act"])
fc_meta["end_act"] = pd.to_datetime(fc_meta["end_act"])
fc_meta = fc_meta[fc_meta["score"] == False]
fc_meta.head()

Score past models:

In [ ]:
act_max = data["period"].max()

fc_archive = pd.read_csv("data/fc48.csv")
fc_archive["period"] = pd.to_datetime(fc_archive["period"])
update_fc_meta = False

for index, row in fc_meta.iterrows():
    if row["start_act"] < act_max:
        label = row["label"]
        n_obs = row["n_obs"]
        f = fc_archive[(fc_archive["label"] == label) & (fc_archive["period"] <= act_max)].merge(data[["period", "value"]], on = "period", how = "left")
        fc_meta.at[index, "mape"] = mean(abs(f["value"] - f["mean"]) / f["value"])
        fc_meta.at[index, "rmse"] = (mean((f["value"] - f["mean"]) ** 2 )) ** 0.5
        fc_meta.at[index, "coverage"] =sum((f["value"] <= f["upper"]) & (f["value"] >= f["lower"])) / len(f)

        if len(f) == n_obs: 
            fc_meta.at[index, "score"] = True
        update_fc_meta = True


In [ ]:
forecast_start = data["period"].max().floor(freq = "d")

In [ ]:
fc_last = fc_meta["end_act"].max()

if fc_last < act_max:
    forecast_start = data["period"].max().floor(freq = "d")
    ts_start = forecast_start - datetime.timedelta(hours = freq * 365 * 2)
    d1 = data[(data["period"] > ts_start) & (data["period"] < forecast_start)]
    ts_obj = pd.DataFrame(np.arange(start = d1["period"].min(), stop = d1["period"].max() + datetime.timedelta(hours = 1), step = datetime.timedelta(hours = 1)).astype(datetime.datetime), columns=["index"])
    ts_obj  = ts_obj.merge(d1, left_on = "index", right_on = "period", how="left")
    input = TimeSeries.from_dataframe(d1,time_col= "period", value_cols= "value")
    lr_model.fit(input)
    lr_preds = lr_model.predict(series=input, 
                            n = h, 
                            num_samples = num_sample )

    lr_preds.plot(label = "Forecast",low_quantile=0.05, high_quantile=0.95)
    pred = lr_preds.pd_dataframe()
    fc = pred.quantile(axis = 1, q = [0.05, 0.5, 0.95]).transpose().reset_index()
    fc = fc.rename(columns = {0.05: "lower", 0.5: "mean", 0.95: "upper"})
    label = str(forecast_start.date())

    fc["label"] = label


    fc_log = {
        "time": datetime.datetime.now(tz=ZoneInfo("UTC")).strftime('%Y-%m-%d %H:%M:%S'),
        "label": label,
        "start": forecast_start,
        "start_act": fc["period"].min(),
        "end_act": fc["period"].max(),
        "h": h,
        "n_obs": len(fc),
        "start_flag": forecast_start == fc["period"].min(),
        "n_obs_flag": h == len(fc),
        "model": "LinearRegressionModel",
        "pi":  0.95,
        "score": False,
        "mape": None,
        "rmse": None,
        "coverage": None
    }

    fc_log["success"] = fc_log["start_flag"] and fc_log["n_obs_flag"]
    
    
    print("Save the forecast metadata into CSV file")
    fc_log_df = pd.DataFrame([fc_log])
    fc_log_new = fc_meta._append(fc_log_df)
    fc_log_new.to_csv("data/fc48_metadata.csv", index = False)
   
    print("Save the forecast into CSV file")
    fc_archive = pd.read_csv("data/fc48.csv")
    fc_archive["period"] = pd.to_datetime(fc_archive["period"])
    new_fc_file = fc_archive._append(fc)
    new_fc_file.to_csv("data/fc48.csv", index = False)
elif update_fc_meta:
    fc_meta.to_csv("data/fc48_metadata.csv", index = False)
